### Ingeniería de características

Equipo:17

Anuar Olmos López A01092551

Arturo Arellano Coyotl A01796088

Mireya Isabel Pérez del Razo A01795608

Profesores
Dra. Grettel Barceló Alonso

Dr. Luis Eduardo Falcón Morales

Fecha: 08/02/2026

## Indice
1.  INTRODUCCIÓN
2.  CARGA Y SPLIT DE DATOS
3.  ANÁLISIS EXPLORATORIO (EDA) + VALIDACIONES
4.  LIMPIEZA Y NORMALIZACIÓN
5.  IMPUTACIÓN DE VALORES FALTANTES
6.  FEATURES NUMÉRICAS DERIVADAS DEL TEXTO
7.  DEFINICIÓN DE FEATURES Y TARGETS
8.  PREPROCESAMIENTO
9.  SELECCIÓN DE FEATURES Y MODELADO
10. ANÁLISIS DE CORRELACIÓN Y PCA/FA
11. CONCLUSIONES

# 1. INTRODUCCIÓN

En este trabajo se desarrolla la fase de ingeniería de características (Feature Engineering) aplicada a una base de datos de comandos en lenguaje natural para el control de una silla de ruedas, donde cada registro combina el estado previo del sistema y una oración (comando) con el estado objetivo que se desea alcanzar. En términos prácticos, la información disponible incluye el estado de acción y el estado de velocidad antes de emitir el comando, así como la oración como un usuario la diría en un escenario real; a partir de ello, el propósito es modelar el “siguiente estado” del sistema, es decir, predecir la acción final y la velocidad final que deberían ejecutarse después de interpretar el comando. Este enfoque es relevante porque en un sistema de asistencia basado en voz, el mismo comando puede depender del contexto (por ejemplo, si ya se encuentra en movimiento o detenido), y porque el lenguaje natural introduce variabilidad real (sinónimos, estilos de escritura, uso de mayúsculas, signos, errores menores o diferencias en acentuación).

El objetivo central de la fase no es únicamente entrenar un modelo, sino transformar datos crudos del mundo real en variables útiles y consistentes para aprendizaje automático. Para lograrlo, primero se realiza una revisión general de la calidad del dataset mediante un análisis exploratorio y validaciones básicas, identificando dimensiones, valores faltantes, duplicados y distribuciones de clase. Después se ejecuta un proceso de limpieza y normalización orientado a reducir ruido: se unifican formatos de texto y categorías, se corrigen inconsistencias comunes de codificación y se estandarizan representaciones para evitar que el modelo aprenda “diferencias” que en realidad son solo variaciones de escritura. Un punto clave es que, en este tipo de datos, es común que algunas etiquetas objetivo no estén explícitas en todos los registros (por ejemplo, si el comando solo cambia la velocidad, la acción puede permanecer igual); por ello se aplica una imputación lógica basada en el estado previo, de manera que el problema refleje correctamente la dinámica del sistema y se mantenga coherencia con el concepto de persistencia del estado cuando no se ordena un cambio.

Posteriormente se construyen nuevas características que complementan el contenido textual. Además de representar el comando con técnicas estándar como TF-IDF (considerando n-gramas de palabras y de caracteres para capturar tanto significado como variaciones de escritura), se generan variables numéricas derivadas de la frase, como longitud del comando, número de palabras, presencia de signos de interrogación o exclamación y otros indicadores que aportan señales adicionales sobre estilo e intención. Se aplican técnicas de escalamiento y transformación (por ejemplo logaritmos y transformaciones tipo Yeo-Johnson) para homogeneizar escalas y favorecer la convergencia de algoritmos lineales, cuidando que cada elección tenga una justificación técnica y esté alineada con el tipo de variable.

Finalmente, se incorporan métodos de selección y extracción de características con el fin de reducir complejidad, tiempo de entrenamiento y requerimientos de almacenamiento, sin sacrificar desempeño. Para esto se evalúan estrategias de filtrado y técnicas de reducción dimensional cuando la representación textual produce espacios de alta dimensionalidad, comparando resultados con métricas adecuadas para clasificación multiclase. A lo largo del trabajo se reporta el impacto de cada decisión sobre el rendimiento y se interpretan los resultados de manera crítica, buscando no solo “obtener una métrica alta”, sino justificar por qué un enfoque es más conveniente para este tipo de datos y para un sistema que, en un escenario real, debe ser robusto, consistente y seguro

# 2. CARGA Y SPLIT DE DATOS

En esta fase se realizó la carga inicial de la base de datos y se dejó lista la partición de entrenamiento y prueba para poder ejecutar el resto del flujo de ingeniería de características de manera ordenada y reproducible. Antes de leer el archivo, se añadió una validación para asegurar que la ruta realmente existe; si no se encuentra el archivo, el código detiene la ejecución y muestra un mensaje claro, evitando que el notebook falle más adelante con errores poco intuitivos. En este caso se observó que la base contiene 3147 registros y 5 columnas.

Después de cargar los datos, se visualizaron las primeras filas para entender rápidamente la estructura. La tabla muestra que cada registro combina el estado previo del sistema (por ejemplo estado_accion y estado_velocidad), la oración del usuario (oracion) y las salidas objetivo (accion y velocidad), que representan el estado final deseado. En el vistazo inicial también se aprecia un comportamiento típico del problema: hay comandos que modifican principalmente la acción (por ejemplo “adelante”) y en esos casos puede aparecer la velocidad como faltante o no explícita; del mismo modo, hay comandos que cambian velocidad mientras la acción se mantiene.

Un punto importante de esta etapa es la forma en que se construyó la partición de entrenamiento y prueba. En lugar de hacer un split aleatorio simple, se creó una variable auxiliar llamada strat combinando estado_velocidad y estado_accion. Con esa columna se realizó un split estratificado, lo que significa que se buscó conservar en ambos conjuntos (train y test) una distribución similar de combinaciones de estados iniciales. 

Finalmente, se separaron explícitamente las variables de entrada y las variables objetivo: como entradas se tomaron estado_accion, estado_velocidad y oracion, mientras que como objetivos se tomaron accion y velocidad. Con esto se preparó la estructura típica de un problema supervisado: X (features) e y (targets). El split se realizó con 80% para entrenamiento y 20% para prueba, obteniendo 2517 filas para entrenamiento y 630 para prueba, y se fijó una semilla (random_state=42) para que el proceso sea reproducible.

In [112]:
# --- (0) Instalación opcional de dependencias ligeras ---
# (Colab normalmente ya trae sklearn; esto solo asegura utilidades para texto)
!pip -q install unidecode

import os
import re
import unicodedata
import numpy as np
import pandas as pd

from unidecode import unidecode

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, PowerTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_classif
from sklearn.decomposition import TruncatedSVD, PCA, FactorAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [113]:
# ============================================
# (2) CARGA Y SPLIT DE DATOS
# ============================================
filename = "../data/raw/Base_de_Datos sucia v7.csv"

if not os.path.exists(filename):
    raise FileNotFoundError(
        f"No encuentro el archivo: {filename}\n"
        "Súbelo a Colab (panel Files) o ajusta la ruta (si está en Drive)."
    )

df = pd.read_csv(filename)
print(" Data cargada:", df.shape)
display(df.head(10))

df['strat']= df['estado_velocidad'] + "_" + df['estado_accion']

feature_cols = ["estado_accion", "estado_velocidad","oracion"]

X = df[feature_cols].copy()
y_accion = df["accion"].copy()
y_velocidad = df["velocidad"].copy()
y_strat = df["strat"].copy()

# Split estratificado por accion (principal), para mantener distribución
X_train, X_test, yA_train, yA_test, yV_train, yV_test = train_test_split(
    X, y_accion, y_velocidad,
    test_size=0.20,
    random_state=42,
    stratify=y_strat
)

print("\n Split listo:",
      "Train:", X_train.shape,
      "Test:", X_test.shape)

print("\n Se guardan dataset de entrenamiento y prueba")
X_train.to_csv("../data/processed/X_train.csv", index=False)
X_test.to_csv("../data/processed/X_val.csv", index=False)
yA_train.to_csv("../data/processed/yA_train.csv", index=False)
yA_test.to_csv("../data/processed/yA_test.csv", index=False)
yV_train.to_csv("../data/processed/yV_train.csv", index=False)
yV_test.to_csv("../data/processed/yV_test.csv", index=False)

 Data cargada: (3147, 5)


,estado_accion,estado_velocidad,oracion,accion,velocidad
0,apagado,detenida,por favor prender,prender,detenida
1,apagado,detenida,prendete rapido,prender,detenida
2,apagado,detenida,enciende,prender,detenida
3,apagado,detenida,activate!,prender,detenida
4,prendida,detenida,Hey Apaguete,apagar,detenida
5,prendida,detenida,descansa ahorita,apagar,detenida
6,prendida,detenida,Oye duerMe por favor,apagar,detenida
7,prendida,detenida,hiberna,apagar,detenida
8,prendida,detenida,adelante,adelante,NaN
9,prendida,detenida,adelante rapido?,adelante,rapido



 Split listo: Train: (2517, 3) Test: (630, 3)

 Se guardan dataset de entrenamiento y prueba


# 3. ANÁLISIS EXPLORATORIO (EDA) + VALIDACIONES

Se retomó el análisis exploratorio de datos (EDA) realizado en la entrega anterior, para poder realizar la ingeniería de características, en este caso solo se incluyeron validaciones básicas de calidad de datos, con la intención de identificar problemas antes de entrar a la ingeniería de características. Se confirmó cuáles son las columnas utilizadas como variables de entrada en el conjunto de entrenamiento (X_train): estado_accion, estado_velocidad y oracion. Esto es importante porque asegura que el modelo trabajará con las tres fuentes de información principales: el contexto previo del sistema (acción y velocidad) y el comando en lenguaje natural. 

In [114]:
# ============================================
# (3) ANÁLISIS RÁPIDO (EDA) + VALIDACIONES
# ============================================
print("\nColumnas:", list(X_train.columns))
print("\nNulos de X train:\n", X_train.isna().sum())
print("\nNulos de Y train acción:\n", yA_train.isna().sum())
print("\nNulos de Y train velocidad:\n", yV_train.isna().sum())
print("\nDuplicados:", X_train.duplicated().sum())

# Distribución de clases (antes de imputaciones)
for col in ["estado_accion", "estado_velocidad", "accion", "velocidad"]:
    if col in X_train.columns:
        print(f"\nTop valores en '{col}':")
        display(X_train[col].value_counts(dropna=False).head(15))


Columnas: ['estado_accion', 'estado_velocidad', 'oracion']

Nulos de X train:
 estado_accion       0
estado_velocidad    0
oracion             0
dtype: int64

Nulos de Y train acción:
 236

Nulos de Y train velocidad:
 480

Duplicados: 74

Top valores en 'estado_accion':


estado_accion
prendida                1529
adelante                 388
apagado                  139
atras                    138
adelante + derecha       121
adelante + izquierda     108
atras + izquierda         24
derecha                   24
atras + derecha           24
izquierda                 22
Name: count, dtype: int64


Top valores en 'estado_velocidad':


estado_velocidad
detenida    1303
normal       448
lenta        400
rapido       366
Name: count, dtype: int64


Se revisó la presencia de valores faltantes (nulos) tanto en las variables de entrada como en las variables objetivo. En las entradas (X_train) el resultado fue muy positivo: no hay nulos en ninguna de las tres columnas (0 en estado_accion, 0 en estado_velocidad, 0 en oracion). Esto significa que no se requiere imputación o limpieza adicional para poder vectorizar el texto o codificar los estados previos. Sin embargo, en las variables objetivo sí se detectaron faltantes: para yA_train (acción objetivo) se encontraron 236 nulos y para yV_train (velocidad objetivo) se encontraron 480 nulos. Esta observación es clave porque confirma que, tal como se esperaba por la naturaleza del problema, existen registros donde el comando no especifica explícitamente un cambio en una de las dos dimensiones (acción o velocidad). Por ejemplo, un comando como “adelante” puede definir claramente la acción final, pero dejar implícita la velocidad (que probablemente se mantiene igual), mientras que un comando como “más rápido” define la velocidad final pero no necesariamente cambia la acción.

Finalmente, se exploró la distribución de clases de los estados previos (estado_accion y estado_velocidad) para entender el contexto inicial con el que más frecuentemente aparece la base. En estado_accion se observa un claro predominio de la categoría “prendida” (1529 casos), seguida por “adelante” (388), “apagado” (139) y “atrás” (138), además de combinaciones como “adelante + derecha” y “adelante + izquierda”. Esto sugiere que gran parte de los registros simulan escenarios donde el sistema ya está encendido y en movimiento, lo cual es consistente con un uso típico de la silla: primero se enciende y luego se ajustan direcciones o velocidad. En estado_velocidad, la clase más frecuente es “detenida” (1303), seguida de “normal” (448), “lenta” (400) y “rapido” (366). Esta distribución indica que muchos comandos se emiten desde reposo o desde un estado de velocidad baja.

# 4. LIMPIEZA Y NORMALIZACIÓN

En esta fase se realizó la limpieza y normalización básica tanto de las etiquetas (estados y targets) como del texto libre de los comandos, con el objetivo de reducir ruido, evitar clases “duplicadas” por diferencias de escritura y dejar una representación consistente antes de generar características como TF-IDF. La justificación principal es que, en datos de lenguaje natural, pequeñas variaciones como acentos, mayúsculas, dobles espacios o caracteres mal decodificados pueden inflar artificialmente el vocabulario y hacer que el modelo aprenda patrones incorrectos. Además, en variables categóricas, cualquier inconsistencia de formato puede crear categorías separadas que en realidad representan lo mismo (por ejemplo, “Prendida”, “prendida ”, “prendída”), afectando directamente la distribución de clases y el aprendizaje.

In [115]:
# ============================================
# (4) LIMPIEZA / NORMALIZACIÓN BÁSICA
# Justificación:
# - Estándares de texto (acentos, espacios, caracteres raros) reducen ruido y sesgos.
# - Etiquetas consistentes evitan clases duplicadas por mayúsculas/espacios.
# ============================================

def fix_mojibake(s: str) -> str:
    """
    Arregla casos típicos tipo 'mÃ¡s' -> 'más' si la cadena viene mal decodificada.
    Si no aplica, devuelve original.
    """
    if not isinstance(s, str):
        return s
    # Heurística: si aparece Ã suele ser mala decodificación UTF-8/Latin1
    if "Ã" in s or "�" in s:
        try:
            return s.encode("latin1").decode("utf-8")
        except Exception:
            return s
    return s

def clean_label(s: str) -> str:
    """
    Limpia etiquetas/categorías:
    - corrige mojibake
    - trim
    - colapsa espacios múltiples
    - baja a minúsculas
    - remueve acentos (para consistencia)
    """
    if not isinstance(s, str):
        return s
    s = fix_mojibake(s)
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    s = s.lower()
    s = unidecode(s)  # quita acentos
    return s

def clean_text_for_tfidf(s: str) -> str:
    """
    Limpieza para features TF-IDF (texto):
    - corrige mojibake
    - minúsculas
    - quita acentos
    - conserva letras/números y signos básicos de intención (? !)
    - colapsa espacios
    """
    if not isinstance(s, str):
        return ""
    s = fix_mojibake(s)
    s = s.strip()
    s = unidecode(s)  # quita acentos
    s = s.lower()
    # conservar letras/numeros, espacios y algunos signos útiles
    s = re.sub(r"[^a-z0-9\s\?\!]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def quitar_stopwords(df, columna):
    """
    Elimina stopwords en español de una columna de texto.
    
    Args:
        df: DataFrame con los datos
        columna: Nombre de la columna a procesar
    
    Returns:
        DataFrame con la columna procesada
    """
    # Stopwords comunes en español
    stopwords_es = {
        'el', 'la', 'de', 'que', 'y', 'a', 'en', 'un', 'ser', 'se', 'haber',
        'por', 'con', 'su',  'como', 'estar', 'tener', 'le', 'lo',
        'pero', 'más', 'hacer', 'o', 'poder', 'decir', 'este', 'ir', 'ese',
        'la', 'si', 'me', 'ya', 'ver', 'porque', 'dar', 'cuando', 'él', 
        'sin', 'vez',  'saber', 'qué', 'sobre', 'mi', 'alguno', 'mismo',
        'yo', 'también', 'año', 'dos', 'querer', 'entre', 'así', 'primero',
        'desde', 'grande', 'eso', 'ni', 'nos', 'llegar', 'pasar', 'tiempo', 'ella',
        'sí', 'día', 'uno', 'bien', 'deber', 'entonces', 'poner', 'cosa',
        'tanto', 'hombre', 'parecer', 'nuestro', 'tan', 'donde', 'ahora', 'parte',
        'después', 'vida', 'siempre', 'creer', 'hablar', 'llevar', 'dejar','cada', 'nuevo', 'encontrar', 
        'decir', 'mundo', 'país', 'contra', 'aquí', 'casa', 'último', 'salir',
        'pregunta', 'trabajar', 'necesitar', 'gobierno', 'número', 'nunca', 'agua',
        'ante', 'cabe', 'cual', 'durante', 'mediante', 'salvo', 'según',
        'excepto', 'hacia', 'tras','ahorita','hey','ahora','que','oye','favor','por','silla'
    }
    
    df_copy = df.copy()
    
    def remover_stopwords(texto):
        if pd.isna(texto):
            return texto
        # Convertir a minúsculas y dividir en palabras
        palabras = texto.lower().split()
        # Filtrar stopwords
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_es]
        return ' '.join(palabras_filtradas)
    
    df_copy[columna] = df_copy[columna].apply(remover_stopwords)
    print(f" Stopwords removidas de la columna '{columna}'")
    
    return df_copy

# Aplicar limpieza a columnas categóricas en X_train y X_test
for c in ["estado_accion", "estado_velocidad"]:
    if c in X_train.columns:
        X_train[c] = X_train[c].apply(clean_label)
        X_test[c] = X_test[c].apply(clean_label)

# Aplicar limpieza a targets
yA_train = yA_train.apply(clean_label)
yA_test = yA_test.apply(clean_label)
yV_train = yV_train.apply(clean_label)
yV_test = yV_test.apply(clean_label)

# Texto original y texto limpio (dos versiones, útil para features extra)
X_train["oracion_raw"] = X_train["oracion"].astype(str)
X_train["oracion_clean"] = X_train["oracion"].apply(clean_text_for_tfidf)
X_train["oracion_clean"] = quitar_stopwords(X_train, "oracion_clean")["oracion_clean"]

X_test["oracion_raw"] = X_test["oracion"].astype(str)
X_test["oracion_clean"] = X_test["oracion"].apply(clean_text_for_tfidf)
X_test["oracion_clean"] = quitar_stopwords(X_test, "oracion_clean")["oracion_clean"]




print("\n Post-limpieza (muestra):")
display(X_train[["estado_accion","estado_velocidad","oracion_raw","oracion_clean"]].head(12))

 Stopwords removidas de la columna 'oracion_clean'
 Stopwords removidas de la columna 'oracion_clean'

 Post-limpieza (muestra):


,estado_accion,estado_velocidad,oracion_raw,oracion_clean
1584,prendida,detenida,silla de reversa mete velocidad,reversa mete velocidad
1332,adelante,lenta,hey a la izquierda,izquierda
1322,prendida,detenida,voltea para la derecha lento,voltea para derecha lento
2956,prendida,rapido,reDuce la Velocidad eN corto,reduce velocidad corto
2921,atras,rapido,oye baja la velocidad,baja velocidad
406,prendida,detenida,dale adelante...,dale adelante
2623,prendida,detenida,hacia adelante rapido,adelante rapido
2331,adelante + derecha,lenta,dobla a la izquierda por favor,dobla izquierda
593,prendida,rapido,echate para atras...,echate para atras
1263,prendida,detenida,silla muevete para atras,muevete para atras


El primer paso fue atender un problema típico cuando se exportan o mueven datos entre sistemas: el “mojibake” o mala decodificación de caracteres. Para eso se definió la función fix_mojibake(), que detecta señales comunes de texto mal interpretado (por ejemplo, la aparición de símbolos como “Ã” o caracteres extraños) y trata de corregirlo re-codificando la cadena. Después se construyó una función de limpieza para etiquetas llamada clean_label(). Esta función se aplica a variables categóricas (estados previos y targets) y tiene varias acciones concretas: corrige mojibake, recorta espacios al inicio/fin (strip), colapsa espacios múltiples a uno, convierte todo a minúsculas y elimina acentos con unidecode. El resultado de esto es que las categorías quedan estandarizadas y no se fragmentan por diferencias superficiales.

Se definió clean_text_for_tfidf(), que está pensada específicamente para preparar el texto que irá a la vectorización TF-IDF. Aquí también se corrige mojibake, se pasa a minúsculas y se eliminan acentos para reducir variabilidad. Además, se aplica un filtrado con expresiones regulares para conservar principalmente letras, números, espacios y algunos signos básicos de intención como “¿?”, “¡!” (que pueden aportar señal en comandos), eliminando caracteres raros que solo añadirían ruido.Un elemento adicional de esta fase fue la eliminación de stopwords en español mediante una lista manual dentro de la función quitar_stopwords(). La lógica aquí es que palabras extremadamente frecuentes como “el”, “de”, “y”, “que”, “para”, etc., suelen aportar poco para diferenciar clases, mientras que sí aumentan el tamaño del vocabulario. Al removerlas, se hace más eficiente la representación TF-IDF y se reduce el ruido. 

En esta sección se conservaron dos versiones del texto: oracion_raw y oracion_clean. La primera mantiene el texto original (útil para revisión y para mostrar evidencia de la transformación), y la segunda contiene el texto normalizado que será usado para la extracción de características. La muestra post-limpieza que se imprime al final confirma que el proceso sí está funcionando: frases como “hey a la izquierda” se simplifican a “izquierda”, “Por Favor Ve Hacia Adelante” se normaliza a “ve adelante”, y “reDuce la Velocidad eN corto” queda como “reduce velocidad corto”. 

# 5. IMPUTACIÓN DE VALORES FALTANTES

En esta fase se abordó uno de los puntos más importantes para que el problema sea consistente: la imputación de valores faltantes en las variables objetivo (targets). A partir del análisis exploratorio previo se observó que, aunque las variables de entrada estaban completas, las salidas objetivo sí contenían valores nulos: algunos registros no tenían definida la acción o la velocidad final. Esto no necesariamente significa un error de captura, sino que es un comportamiento esperado cuando se trabaja con comandos en lenguaje natural. En la práctica, hay frases que se enfocan únicamente en modificar la velocidad (“más rápido”, “más lento”, “normal”), por lo que la acción final puede quedar implícita; del mismo modo, existen frases que cambian la acción (“adelante”, “izquierda”, “derecha”, “atrás”), donde la velocidad final no siempre se especifica explícitamente porque se asume que se mantiene como estaba o que sigue una regla de operación.

In [116]:
# ============================================
# (5) IMPUTACIÓN DE VALORES FALTANTES
# Observación del dataset:
# - Cuando el comando es de VELOCIDAD, 'accion' suele venir NaN.
# - Cuando el comando es de ACCIÓN (mover/girar), 'velocidad' suele venir NaN.
# Justificación:
# - Para un modelo de "próximo estado", si no se menciona el cambio, se asume que permanece igual.
# ============================================

# Rellenar NaN en targets usando el estado previo:
# Si no cambia acción -> acción final = estado_accion

yA_train = yA_train.replace('', np.nan)
yV_train = yV_train.replace('', np.nan)
yA_test = yA_test.replace('', np.nan)
yV_test = yV_test.replace('', np.nan)

#Imputación  de la acción tomando como base el estado anterior
yA_train = yA_train.fillna(X_train["estado_accion"])
yV_train = yV_train.fillna("normal")

#Imputación  de la velocidad, se establece velocidad normal
yA_test = yA_test.fillna(X_test["estado_accion"])
yV_test = yV_test.fillna("normal")

print("\n Nulos de Y train acción:", yA_train.isna().sum())
print(" Nulos de Y train velocidad:", yV_train.isna().sum())


 Nulos de Y train acción: 0
 Nulos de Y train velocidad: 0


La decisión de imputar estos valores se justificó bajo un enfoque de “próximo estado” (next state). En un sistema real, si el usuario no menciona que quiere cambiar una dimensión del estado, se interpreta que esa dimensión permanece igual. Es decir, el comando modifica solo lo que se declara explícitamente y lo demás se conserva. Por eso, el enfoque utilizado fue rellenar los faltantes en los targets siguiendo una regla lógica basada en el estado previo. Primero, se hizo una limpieza preventiva reemplazando cadenas vacías por valores NaN, porque en algunos archivos los faltantes pueden venir como texto vacío y eso puede impedir que fillna() funcione correctamente. Esto asegura que todos los “faltantes” se traten de forma uniforme.

Después, se imputó la acción final (yA_train) con el valor de estado_accion cuando la acción venía nula. Esto significa que, si un comando no está orientado a cambiar la acción, se asume que la acción final se mantiene igual que la acción en la que ya estaba el sistema.Este paso funciona con comandos centrados en velocidad o en confirmaciones (“rápido”, “normal”, etc.), donde lo lógico es conservar el movimiento o dirección actual y solo ajustar la rapidez.

Para la velocidad final (yV_train), en este fragmento se imputaron los faltantes con el valor “normal”. Esta es una decisión práctica que puede interpretarse como un valor “por defecto” cuando el comando no especifica velocidad. En términos de operación, “normal” suele representar una velocidad estándar segura. Finalmente, se validó el resultado de la imputación confirmando que ya no existen valores faltantes en los targets: el conteo de nulos para yA_train y yV_train pasó a cero. Esto es un requisito esencial para poder entrenar cualquier modelo supervisado, porque la mayoría de algoritmos no admiten etiquetas objetivo faltantes.

# 6. FEATURES NUMÉRICAS DERIVADAS DEL TEXTO

En esta sección se realizó la generación de características numéricas derivadas del texto, es decir, se transformó cada oración (comando) en un conjunto de variables cuantitativas simples que capturan información adicional que a veces el TF-IDF no refleja de manera directa. La idea detrás de esto es que, además del “contenido” de las palabras, el estilo del comando (su longitud, si incluye signos, si está escrito con mayúsculas, si tiene tono de urgencia o cortesía, etc.) puede aportar señales complementarias para distinguir patrones asociados a ciertas acciones o cambios de velocidad. Este paso también es útil porque permite después aplicar discretización (binning) y transformaciones como logaritmos o Yeo-Johnson sobre variables numéricas.

In [117]:
# ============================================
# (6) FEATURES NUMÉRICAS DERIVADAS DEL TEXTO
# Incluye:
# - longitud de caracteres, #palabras, #signos ?,!, proporción mayúsculas, etc.
# Justificación:
# - Aporta señales complementarias al TF-IDF (intención, énfasis, estilo de comando).
# - Permite discretización/binning y transformaciones (log, Yeo-Johnson).
# ============================================

def text_stats(raw: str) -> dict:
    if not isinstance(raw, str):
        raw = ""
    s = fix_mojibake(raw)
    s_strip = s.strip()
    words = re.findall(r"\b\w+\b", s_strip, flags=re.UNICODE)
    n_words = len(words)
    n_chars = len(s_strip)
    n_q = s_strip.count("?")
    n_ex = s_strip.count("!")
    n_digits = sum(ch.isdigit() for ch in s_strip)
    n_upper = sum(ch.isupper() for ch in s_strip)
    n_alpha = sum(ch.isalpha() for ch in s_strip)
    upper_ratio = (n_upper / n_alpha) if n_alpha > 0 else 0.0
    has_polite = int(bool(re.search(r"\b(por\s+favor|porfavor|please|pls)\b", s_strip, flags=re.IGNORECASE)))
    has_urgent = int(bool(re.search(r"\b(ahorita|ya|de\s+inmediato|inmediato)\b", s_strip, flags=re.IGNORECASE)))
    return {
        "n_chars": n_chars,
        "n_words": n_words,
        "n_qmarks": n_q,
        "n_excl": n_ex,
        "n_digits": n_digits,
        "upper_ratio": upper_ratio,
        "has_polite": has_polite,
        "has_urgent": has_urgent,
    }

# Aplicar a X_train
stats_df_train = X_train["oracion_raw"].apply(text_stats).apply(pd.Series)
X_train = pd.concat([X_train, stats_df_train], axis=1)

# Aplicar a X_test
stats_df_test = X_test["oracion_raw"].apply(text_stats).apply(pd.Series)
X_test = pd.concat([X_test, stats_df_test], axis=1)

# Discretización / binning (ejemplo):
# - Bins para longitud de frase (corta/mediana/larga)
X_train["len_bin"] = pd.cut(X_train["n_chars"], bins=[-1, 8, 18, 9999], labels=["corta","mediana","larga"])
X_test["len_bin"] = pd.cut(X_test["n_chars"], bins=[-1, 8, 18, 9999], labels=["corta","mediana","larga"])

print("\n Features numéricas creadas:")
display(X_train[["oracion_raw","n_chars","n_words","n_qmarks","n_excl","upper_ratio","has_polite","has_urgent","len_bin"]].head(15))


 Features numéricas creadas:


,oracion_raw,n_chars,n_words,n_qmarks,n_excl,upper_ratio,has_polite,has_urgent,len_bin
1584,silla de reversa mete velocidad,31.0,5.0,0.0,0.0,0.000000,0.0,0.0,larga
1332,hey a la izquierda,18.0,4.0,0.0,0.0,0.000000,0.0,0.0,mediana
1322,voltea para la derecha lento,28.0,5.0,0.0,0.0,0.000000,0.0,0.0,larga
2956,reDuce la Velocidad eN corto,28.0,5.0,0.0,0.0,0.125000,0.0,0.0,larga
2921,oye baja la velocidad,21.0,4.0,0.0,0.0,0.000000,0.0,0.0,larga
406,dale adelante...,16.0,2.0,0.0,0.0,0.000000,0.0,0.0,mediana
2623,hacia adelante rapido,21.0,3.0,0.0,0.0,0.000000,0.0,0.0,larga
2331,dobla a la izquierda por favor,30.0,6.0,0.0,0.0,0.000000,1.0,0.0,larga
593,echate para atras...,20.0,3.0,0.0,0.0,0.000000,0.0,0.0,larga
1263,silla muevete para atras,24.0,4.0,0.0,0.0,0.000000,0.0,0.0,larga


Se definió la función text_stats(raw), que toma la oración original (oracion_raw) y calcula varios indicadores. Primero se asegura que el texto sea realmente una cadena; si no lo es, se maneja el caso de forma segura. Luego se vuelve a utilizar la corrección de mojibake para evitar que una mala decodificación altere conteos o tokens. A partir de la cadena ya corregida se calculan métricas como: número de caracteres (n_chars), número de palabras (n_words), cantidad de signos de interrogación (n_qmarks) y cantidad de signos de exclamación (n_excl). También se cuenta el número de dígitos (n_digits) y se estima una señal de “énfasis” en escritura mediante upper_ratio, que es la proporción de letras en mayúscula respecto al total de letras (esto captura casos como comandos escritos “FUERTE” o con mezcla extraña de mayúsculas/minúsculas). Aunque estas métricas son simples, suelen ser útiles porque reflejan variabilidad real del lenguaje.

Además, se incluyeron dos indicadores binarios muy orientados al dominio del dataset: has_polite y has_urgent. El primero detecta cortesía mediante expresiones como “por favor”, “please”, “pls”, etc.; el segundo detecta urgencia con términos como “ahorita”, “ya”, “de inmediato”, “inmediato”, etc. “por favor” es común pero no cambia el significado central; aun así, su presencia puede correlacionarse con ciertos tipos de comandos o con oraciones más largas, y ayuda a capturar diferencias de estilo. En otras palabras, no se espera que “por favor” cambie la acción en sí, pero sí puede ayudar a que el modelo sea más robusto y a que no confunda patrones por ruido de lenguaje.

Una vez calculadas estas métricas para cada fila, se aplicó la función sobre las oraciones de entrenamiento y prueba, y el resultado se convirtió en un DataFrame. Después, esas nuevas columnas se concatenaron a los datos originales, con lo que tanto el entrenamiento como prueba quedaron enriquecidos con estas nuevas variables numéricas. Asimismo, se realizó un ejemplo explícito de discretización (binning) usando la longitud en caracteres (n_chars). Se crearon rangos para clasificar cada comando como “corta”, “mediana” o “larga” con límites predefinidos. Esta discretización puede ser útil para capturar patrones por intervalos (por ejemplo, comandos muy cortos como “izquierda” o “adelante” frente a comandos largos con varias palabras) y también aporta una variable categórica adicional que posteriormente puede codificarse con one-hot.

# 7. DEFINICION DE FEATURES Y TARGETS

Se definen cuáles son las variables de entrada (features) finales y cuáles son las variables objetivo (targets). Esto evita muchísimos errores, porque aquí es donde se decide exactamente qué información se le proporcionará al modelo y qué es lo que se va a predecir. Además con esto se muestran las variables iniciales a usar y las que se van a generar.

In [118]:
# ============================================
# (7) DEFINICIÓN DE FEATURES
# ============================================

feature_cols = ["estado_accion", "estado_velocidad", "oracion_clean",
                "n_chars","n_words","n_qmarks","n_excl","n_digits","upper_ratio","has_polite","has_urgent","len_bin"]

print("\n Features definidas para el modelo:")
print(feature_cols)
print("\nDimensiones actuales:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("yA_train:", yA_train.shape[0])
print("yV_train:", yV_train.shape[0])


 Features definidas para el modelo:
['estado_accion', 'estado_velocidad', 'oracion_clean', 'n_chars', 'n_words', 'n_qmarks', 'n_excl', 'n_digits', 'upper_ratio', 'has_polite', 'has_urgent', 'len_bin']

Dimensiones actuales:
X_train: (2517, 14)
X_test: (630, 14)
yA_train: 2517
yV_train: 2517


Primero, se creó la lista feature_cols, que incluye tres tipos de información: (1) contexto del sistema antes del comando (estado_accion y estado_velocidad), (2) texto ya normalizado (oracion_clean), y (3) características numéricas derivadas del texto (las que se generaron en el paso anterior: n_chars, n_words, n_qmarks, n_excl, n_digits, upper_ratio, has_polite, has_urgent, además de la variable discretizada len_bin). En total, esta definición da 14 columnas de entrada. La idea detrás de esta combinación es que el modelo no dependa únicamente del texto: se le da también el contexto previo (muy importante para interpretar comandos) y señales cuantitativas del estilo del comando, que complementan lo semántico.

Luego se imprimieron dos cosas para validación: primero, el listado completo de features para confirmar que efectivamente están las columnas esperadas; y segundo, las dimensiones actuales de los datasets. Aquí se observa que X_train quedó con forma (2517, 14), mientras que X_test quedó con forma (630, 14). Esto confirma dos puntos clave: que las nuevas variables se agregaron correctamente, y que train y test tienen exactamente el mismo número de features.

También se imprimieron las dimensiones de los targets: yA_train y yV_train aparecen con longitud (2517), lo que indica que por cada fila de X_train existe una etiqueta para acción y una etiqueta para velocidad. Antes de la imputación lógica, esos targets tenían valores faltantes; aquí ya se valida indirectamente que el dataset quedó coherente: mismo número de registros en X y en y, y por lo tanto listo para entrenar modelos supervisados.

# 8. PREPROCESAMIENTO

En esta parte se construyó el preprocesamiento completo que convierte las columnas finales (categóricas, numéricas y texto) en una matriz numérica lista para alimentar modelos de Machine Learning. La idea aquí es que cada tipo de dato necesita un tratamiento distinto: las categorías no se pueden dejar como texto, las variables numéricas suelen requerir escalamiento/transformaciones para que no dominen por magnitud, y el texto se debe vectorizar para transformarlo en features cuantificables. Por eso se diseñó un pipeline modular y reproducible usando Pipeline y ColumnTransformer, que además evita errores comunes como aplicar transformaciones distintas en entrenamiento y prueba.

In [ ]:
# ============================================
# (8) PREPROCESAMIENTO (ENCODING, ESCALADO, TRANSFORMACIONES)
# ============================================

cat_onehot_features = ["estado_accion", "len_bin"]
cat_ordinal_features = ["estado_velocidad"]

num_features = [
    "n_chars","n_words","n_qmarks","n_excl","n_digits","upper_ratio","has_polite","has_urgent"
]

text_feature = "oracion_clean"

# =============================
# TRANSFORMACIÓN NUMÉRICA
# =============================
def log1p_safe(Xn):
    Xn = np.asarray(Xn, dtype=float)
    return np.log1p(np.clip(Xn, 0, None))

numeric_pipeline = Pipeline(steps=[
    ("log1p", FunctionTransformer(log1p_safe, feature_names_out="one-to-one")),
    ("power", PowerTransformer(method="yeo-johnson", standardize=True))
])

# =============================
# TRANSFORMACIÓN CATEGÓRICA
# =============================

# One-Hot para categóricas sin orden
categorical_onehot = OneHotEncoder(handle_unknown="ignore")

# Ordinal Encoding para estado_velocidad
ordinal_velocity = OrdinalEncoder(
    categories=[["detenida", "lento", "normal", "rapido"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

# =============================
# TF-IDF
# =============================
tfidf_word = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_features=25000
)

tfidf_char = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    max_features=25000
)

# =============================
# COLUMN TRANSFORMERS
# =============================
preprocess_word = ColumnTransformer(
    transformers=[
        ("cat_oh", categorical_onehot, cat_onehot_features),
        ("cat_ord", ordinal_velocity, cat_ordinal_features),
        ("num", numeric_pipeline, num_features),
        ("txt", tfidf_word, text_feature),
    ],
    remainder="drop"
)

preprocess_word_char = ColumnTransformer(
    transformers=[
        ("cat_oh", categorical_onehot, cat_onehot_features),
        ("cat_ord", ordinal_velocity, cat_ordinal_features),
        ("num", numeric_pipeline, num_features),
        ("txtw", tfidf_word, text_feature),
        ("txtc", tfidf_char, text_feature),
    ],
    remainder="drop"
)

print("Pipelines de preprocesamiento definidos")

# =============================
# APLICAR PREPROCESAMIENTO
# =============================
X_train_prep = preprocess_word_char.fit_transform(X_train[feature_cols])
X_test_prep = preprocess_word_char.transform(X_test[feature_cols])

print(f"\n Transformaciones aplicadas exitosamente")
print(f"\n DIMENSIONES FINALES:")
print(f"   X_train_prep: {X_train_prep.shape}")
print(f"   X_test_prep:  {X_test_prep.shape}")
print(f"   yA_train:     {yA_train.shape}")
print(f"   yV_train:     {yV_train.shape}")

print(f"\n COMPOSICIÓN DE FEATURES:")
print(f"   - One-Hot: Se transformaron {len(cat_onehot_features)} columnas categóricas")
print(f"   - Ordinal: Se transformaron {len(cat_ordinal_features)} columnas categóricas")
print(f"   - Numéricas: Se transformaron {len(num_features)} columnas")
print(f"   - Texto (TF-IDF palabras + caracteres)")
print(f"   - TOTAL: {X_train_prep.shape[1]} features")

print(f"\n TIPO DE DATOS:")
print(f"   Tipo: {type(X_train_prep)}")
print(f"   Formato: Matriz dispersa (sparse)")

if hasattr(X_train_prep, 'nnz'):
    density = (X_train_prep.nnz / (X_train_prep.shape[0] * X_train_prep.shape[1])) * 100
    print(f"\n ESTADÍSTICAS:")
    print(f"   Elementos no-cero: {X_train_prep.nnz:,}")
    print(f"   Densidad: {density:.4f}%")
    print(f"   Esparsidad: {100-density:.4f}%")

# =============================
# MUESTRA PARA INSPECCIÓN
# =============================
print(f"\n Primeras 20 columnas transformadas")
X_train_dense_sample = X_train_prep[:5, :20].toarray()
sample_df = pd.DataFrame(
    X_train_dense_sample,
    columns=[f"feature_{i}" for i in range(20)]
)
display(sample_df)

print(f"\n Datos preprocesados y listos para entrenamiento")

Pipelines de preprocesamiento definidos

 Transformaciones aplicadas exitosamente

 DIMENSIONES FINALES:
   X_train_prep: (2517, 1914)
   X_test_prep:  (630, 1914)
   yA_train:     (2517,)
   yV_train:     (2517,)

 COMPOSICIÓN DE FEATURES:
   - One-Hot: Se transformaron 2 columnas categóricas
   - Ordinal: Se transformaron 1 columnas categóricas
   - Numéricas: Se transformaron 8 columnas
   - Texto (TF-IDF palabras + caracteres)
   - TOTAL: 1914 features

 TIPO DE DATOS:
   Tipo: <class 'scipy.sparse._csr.csr_matrix'>
   Formato: Matriz dispersa (sparse)

 ESTADÍSTICAS:
   Elementos no-cero: 126,347
   Densidad: 2.6226%
   Esparsidad: 97.3774%

 Primeras 20 columnas transformadas


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.764525,0.495282,-0.261412,-0.298561,0.0,-0.593545
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,-0.655156,-0.109258,-0.261412,-0.298561,0.0,-0.593545
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.461364,0.495282,-0.261412,-0.298561,0.0,-0.593545
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.461364,0.495282,-0.261412,-0.298561,0.0,1.152766
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,-0.300637,-0.109258,-0.261412,-0.298561,0.0,-0.593545



 Datos preprocesados y listos para entrenamiento


Se definieron explícitamente los grupos de columnas según su naturaleza. Las variables categóricas se dividieron en dos subconjuntos: estado_accion y len_bin, que no presentaban un orden semántico, se codificaron mediante One-Hot Encoding; estado_velocidad, que sí posee un orden natural (detenida < lento < normal < rápido)  se transformó mediante Ordinal Encoding. Las variables numéricas corresponden a métricas derivadas del texto (n_chars, n_words, n_qmarks, n_excl, n_digits, upper_ratio, has_polite, has_urgent). Finalmente, el texto principal se representó mediante la columna oracion_clean, previamente normalizada y depurada.

Para las variables numéricas se construyó un pipeline con dos etapas. Se aplicó una transformación log1p segura sobre variables de conteo, posteriormente se utilizó PowerTransformer con el método Yeo-Johnson y estandarización, dejando distribuciones más simétricas y a las variables con una  escala comparable, lo que favorece la convergencia de modelos lineales.

Para el texto se utilizaron dos representaciones TF-IDF complementarias. La primera, basada en palabras con unigramas y bigramas, permite capturar tanto términos individuales como combinaciones frecuentes que aportan contexto semántico. La segunda, basada en n-gramas de caracteres, resulta más robusta ante variaciones ortográficas y errores simples de escritura, algo común en comandos escritos por usuarios. Con estos componentes se definieron dos ColumnTransformer: uno que combina variables categóricas, numéricas y TF-IDF por palabras, y otro que añade además TF-IDF por caracteres.

Al aplicar el pipeline que combina TF-IDF por palabras y caracteres junto con codificación ordinal para la velocidad, el conjunto de entrenamiento quedó representado por una matriz dispersa de dimensiones (2517, 1889) y el conjunto de prueba por (630, 1889). 

La matriz resultante es altamente dispersa (≈ 97.4 % de ceros), lo cual es característico de representaciones TF-IDF y deseable desde el punto de vista computacional. Este formato es eficiente en memoria y adecuado para modelos lineales, SVMs y métodos basados en regularización, que suelen desempeñarse bien en espacios de alta dimensionalidad.

# 9. SELECCIÓN DE FEATURES Y MODELADO

En la sección anterior se concluyó con la ingeniería de características; sin embargo, para verificar que estás sean útiles para el entrenamiento de un modelo, se realizó una comparación para verificar si al “reducir” o “compactar” las features se mantiene o mejora el desempeño del modelo. La lógica general es: primero se entrena un modelo base sin selección explícita (usando todas las features generadas por el preprocesamiento), y después se prueba una alternativa donde se aplica reducción dimensional (TruncatedSVD) para “extraer” un número menor de componentes y así reducir la complejidad, y el tiempo de procesamiento.

In [120]:
# ============================================
# (9) SELECCIÓN DE FEATURES Y MODELADO
# ============================================

# Modelo base (sin selección explícita): LinearSVC suele ir muy bien con TF-IDF
model_base = LinearSVC(class_weight="balanced", max_iter=3000)

pipe_accion_base = Pipeline(steps=[
    ("prep", preprocess_word_char),
    ("clf", model_base)
])

pipe_vel_base = Pipeline(steps=[
    ("prep", preprocess_word_char),
    ("clf", model_base)
])

print("\n==============================")
print("MODELO BASE (sin selección)")
print("==============================")

pipe_accion_base.fit(X_train, yA_train)
predA = pipe_accion_base.predict(X_test)
print("\n ACCIÓN - F1 macro:", round(f1_score(yA_test, predA, average="macro"), 4))
print(classification_report(yA_test, predA))

pipe_vel_base.fit(X_train, yV_train)
predV = pipe_vel_base.predict(X_test)
print("\n VELOCIDAD - F1 macro:", round(f1_score(yV_test, predV, average="macro"), 4))
print(classification_report(yV_test, predV))

# Selección por reducción dimensional (TruncatedSVD)
svd = TruncatedSVD(n_components=250, random_state=42)

pipe_accion_svd = Pipeline(steps=[
    ("prep", preprocess_word_char),
    ("var", VarianceThreshold(threshold=0.0)),
    ("svd", svd),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", n_jobs=-1))
])

pipe_vel_svd = Pipeline(steps=[
    ("prep", preprocess_word_char),
    ("var", VarianceThreshold(threshold=0.0)),
    ("svd", svd),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", n_jobs=-1))
])

print("\n==============================")
print("MODELO CON SVD (reducción dimensional)")
print("==============================")

pipe_accion_svd.fit(X_train, yA_train)
predA2 = pipe_accion_svd.predict(X_test)
print("\n ACCIÓN (SVD) - F1 macro:", round(f1_score(yA_test, predA2, average="macro"), 4))
print(classification_report(yA_test, predA2))

pipe_vel_svd.fit(X_train, yV_train)
predV2 = pipe_vel_svd.predict(X_test)
print("\n VELOCIDAD (SVD) - F1 macro:", round(f1_score(yV_test, predV2, average="macro"), 4))
print(classification_report(yV_test, predV2))


MODELO BASE (sin selección)

 ACCIÓN - F1 macro: 0.9541
                      precision    recall  f1-score   support

            adelante       0.99      0.95      0.97       184
  adelante + derecha       0.94      1.00      0.97        33
adelante + izquierda       1.00      0.91      0.95        43
              apagar       1.00      1.00      1.00        35
               atras       0.99      0.94      0.96       146
     atras + derecha       1.00      1.00      1.00         3
   atras + izquierda       1.00      1.00      1.00         2
             derecha       1.00      0.90      0.95        40
           izquierda       0.86      0.89      0.88        36
             prender       1.00      1.00      1.00        52
            prendida       0.70      0.96      0.81        56

            accuracy                           0.95       630
           macro avg       0.95      0.96      0.95       630
        weighted avg       0.96      0.95      0.95       630


 VELOCIDA

En el modelo base se armó un Pipeline con dos partes: (1) preprocess_word_char y (2) un clasificador LinearSVC(class_weight="balanced", max_iter=3000). El parámetro class_weight="balanced" se utiliza para compensar el desbalance entre clases y evitar que el modelo favorezca sistemáticamente a las más frecuentes.

Se entrenaron dos modelos independientes con este pipeline base: uno para predecir "accion" (yA_train) y otro para predecir VELOCIDAD (yV_train). La métrica principal reportada es F1 macro, adecuada en este contexto porque promedia el desempeño por clase dándole el mismo peso a todas, independientemente de su frecuencia.

En los resultados del modelo base se observa un desempeño muy alto. Para "accion" se alcanza un F1 macro = 0.9541, con la mayoría de las clases mostrando valores de F1 cercanos o superiores a 0.95. Clases como adelante, atrás, apagar y prender presentan un desempeño prácticamente perfecto. Las clases más difíciles siguen siendo izquierda (F1 ≈ 0.88) y prendida (F1 ≈ 0.81).

Para"velocidad", el modelo base obtiene un F1 macro = 0.9451. Las clases detenida y lenta presentan un desempeño muy alto (F1 ≈ 0.99–0.95), mientras que normal baja a (F1 ≈ 0.93), probablemente debido a que suele aparecer en contextos más ambiguos o implícitos. La clase rapido muestra el F1 más bajo (≈ 0.91), con alta recall pero menor precisión, lo que indica que el modelo tiende a sobrepredecir esta clase cuando detecta términos relacionados con aceleración.

Posteriormente se evaluó el modelo con SVD (reducción dimensional), agregando VarianceThreshold para eliminar features constantes, y luego TruncatedSVD con 250 componentes, que actúa como una especie de PCA adaptado a matrices dispersas. Finalmente, se utilizó LogisticRegression como clasificador lineal.

Al comparar resultados, se observó una ligera caída en desempeño respecto al modelo base. Para "accion", el F1 macro baja a 0.9533. Las clases más complejas (izquierda y prendida) siguen siendo las que más contribuyen a esta caída. En "velocidad", el F1 macro desciende a 0.9, con una reducción más notable en la clase "normal" y un patrón similar al del modelo base: "rapido" mantiene alto recall pero menor precisión.

# 10. ANÁLISIS DE CORRELACIÓN y PCA/FA

Se hizo un análisis sobre las variables numéricas que se generaron a partir del texto (por ejemplo: longitud en caracteres, número de palabras, signos de interrogación/exclamación, proporción de mayúsculas, banderas de cortesía/urgencia, etc.). Es conveniente revisar si algunas de esas variables tienen alta correlación, porque eso puede generar ruido y multicolinealidad y hacer el entrenamiento menos estable; además con esto se evalua si se pueden reducir dimensiones con técnicas como PCA o FA sin perder demasiada información.

In [121]:
# ============================================
# (10) ANÁLISIS DE CORRELACIÓN Y PCA/FA
# ============================================

# Análisis de correlación en features numéricas
num_features_for_corr = ["n_chars","n_words","n_qmarks","n_excl","n_digits","upper_ratio","has_polite","has_urgent"]

num_corr = X_train[num_features_for_corr].corr(numeric_only=True).abs()
print("Matriz de correlación de features numéricas:")
display(num_corr)

# Quitar features altamente correlacionadas entre sí (ej. >0.95)
upper = num_corr.where(np.triu(np.ones(num_corr.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
print("\nFeatures numéricas altamente correlacionadas para considerar remover:", to_drop)

# PCA / Factor Analysis en bloque numérico
num_pipe_only = Pipeline(steps=[
    ("log1p", FunctionTransformer(lambda X: np.log1p(np.clip(X, 0, None)), feature_names_out="one-to-one")),
    ("power", PowerTransformer(method="yeo-johnson", standardize=True))
])

Xnum_train = X_train[num_features_for_corr].values
Xnum_train_t = num_pipe_only.fit_transform(Xnum_train)

pca = PCA(n_components=3, random_state=42)
Xnum_pca = pca.fit_transform(Xnum_train_t)

fa = FactorAnalysis(n_components=3, random_state=42)
Xnum_fa = fa.fit_transform(Xnum_train_t)

print("\nPCA explained variance ratio (3 comps):", pca.explained_variance_ratio_)
print("PCA cumulative:", np.cumsum(pca.explained_variance_ratio_))

# Guardar componentes como features nuevas (opcional)
X_train_pca = X_train.copy()
X_train_pca["pca1"], X_train_pca["pca2"], X_train_pca["pca3"] = Xnum_pca[:,0], Xnum_pca[:,1], Xnum_pca[:,2]
X_train_pca["fa1"], X_train_pca["fa2"], X_train_pca["fa3"] = Xnum_fa[:,0], Xnum_fa[:,1], Xnum_fa[:,2]

# Aplicar transformación a X_test también
Xnum_test = X_test[num_features_for_corr].values
Xnum_test_t = num_pipe_only.transform(Xnum_test)
Xnum_test_pca = pca.transform(Xnum_test_t)
Xnum_test_fa = fa.transform(Xnum_test_t)

X_test_pca = X_test.copy()
X_test_pca["pca1"], X_test_pca["pca2"], X_test_pca["pca3"] = Xnum_test_pca[:,0], Xnum_test_pca[:,1], Xnum_test_pca[:,2]
X_test_pca["fa1"], X_test_pca["fa2"], X_test_pca["fa3"] = Xnum_test_fa[:,0], Xnum_test_fa[:,1], Xnum_test_fa[:,2]

print("\nComponentes PCA/FA agregados al dataset:")
display(X_train_pca[["pca1","pca2","pca3","fa1","fa2","fa3"]].head())

Matriz de correlación de features numéricas:


,n_chars,n_words,n_qmarks,n_excl,n_digits,upper_ratio,has_polite,has_urgent
n_chars,1.000000,0.923652,0.035857,0.017795,NaN,0.013044,0.300244,0.262234
n_words,0.923652,1.000000,0.009077,0.029949,NaN,0.000793,0.346104,0.221247
n_qmarks,0.035857,0.009077,1.000000,0.070095,NaN,0.026222,0.014592,0.014251
n_excl,0.017795,0.029949,0.070095,1.000000,NaN,0.020506,0.006445,0.014112
n_digits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
upper_ratio,0.013044,0.000793,0.026222,0.020506,NaN,1.000000,0.012757,0.012326
has_polite,0.300244,0.346104,0.014592,0.006445,NaN,0.012757,1.000000,0.097181
has_urgent,0.262234,0.221247,0.014251,0.014112,NaN,0.012326,0.097181,1.000000



Features numéricas altamente correlacionadas para considerar remover: []

PCA explained variance ratio (3 comps): [0.31160505 0.15703708 0.15406085]
PCA cumulative: [0.31160505 0.46864213 0.62270298]

Componentes PCA/FA agregados al dataset:


,pca1,pca2,pca3,fa1,fa2,fa3
1584,0.497089,-0.214555,0.177071,0.614208,-0.086808,-0.119285
1332,-0.808706,-0.272176,0.202874,-0.405429,-0.220046,0.307117
1322,0.301709,-0.227313,0.182463,0.458107,-0.137586,0.027546
2956,0.273244,-0.061093,-0.109224,0.457060,-0.176043,0.027512
2921,-0.580228,-0.257256,0.196568,-0.222883,-0.160666,0.135411


Primero se calculó una matriz de correlación (en valor absoluto) entre las variables numéricas: n_chars, n_words, n_qmarks, n_excl, n_digits, upper_ratio, has_polite, has_urgent. Aquí el resultado más importante es que n_chars y n_words salen fuertemente correlacionadas (~0.9236). Esto es totalmente esperable: a mayor número de palabras, normalmente también crece el número de caracteres. Aun así, como el umbral que se definió para considerar una variable “redundante” fue 0.95, esa correlación no alcanzó el nivel para eliminar una de las dos automáticamente.

El resto de pares muestran correlaciones bajas ( n_qmarks, n_excl, upper_ratio,has_polite y has_urgent). Un punto clave del output es que n_digits aparece con NaN en la correlación, esto normalmente pasa cuando la variable tiene varianza cero, y entonces la correlación no se puede calcular. Con este análisis se concluye que no hay features numéricas “duplicadas” por encima del umbral, y por eso el listado de features para remover salió vacío.

Después se aplicó PCA (Análisis de Componentes Principales) únicamente al bloque numérico. Antes de PCA se hizo una transformación pensada para datos tipo conteo y variables con asimetría: se aplicó log1p (para estabilizar variables de conteo como n_words, n_chars, etc., evitando que los valores grandes dominen) y luego un PowerTransformer con Yeo-Johnson y estandarización (para llevarlas a una escala comparable y con una distribución más cercana a lanormal).

Al aplicar PCA de 3 componentes, los resultados indican varianzas explicadas aproximadas de 0.3116, 0.1570 y 0.1541; en conjunto, las tres componentes acumulan 62,3%. Esto signidica que con solo 3 variables (pca1, pca2, pca3) se resume mas del 60% de la variabilidad del bloque numérico. 

En paralelo se calculó FA (Factor Analysis) con 3 factores (fa1, fa2, fa3) usando el mismo bloque numérico ya transformado. FA se parece a PCA en que reduce dimensionalidad, pero conceptualmente busca explicar la covarianza a través de “factores latentes” (señales subyacentes compartidas), mientras que PCA prioriza explicar la varianza total. 

# 11. CONCLUSIONES

El desarrollo de la fase de ingeniería de características confirmó la importancia de un tratamiento cuidadoso de datos contextuales y textuales en sistemas de asistencia basados en voz. A lo largo del proceso se evidenció que el significado de un comando no puede interpretarse de forma aislada, sino que depende fuertemente del estado previo del sistema (acción y velocidad), lo que valida el enfoque de modelar el problema como una predicción de “siguiente estado”.

En la etapa de carga y partición de datos, se verificó la consistencia del dataset tanto en tamaño como en estructura, y se justificó el uso de un split estratificado basado en combinaciones de estados previos. Esta decisión resultó fundamental para evitar sesgos de evaluación, dado que el contexto inicial influye directamente en la interpretación del comando.

El análisis exploratorio (EDA) permitió identificar desde etapas tempranas dos aspectos críticos: por un lado, que las variables de entrada estaba completas, y por otro, la presencia de valores faltantes en las variables objetivo. Esta observación justificó la imputación lógica de datos tomando en cuenta el estado previo.

La fase de limpieza y normalización redujo significativamente el ruido introducido por variaciones superficiales del lenguaje natural y por inconsistencias en etiquetas categóricas. Mantener tanto el texto original como el texto limpio reforzó la trazabilidad del proceso y facilita la revisión de las transformaciones aplicadas.

En cuanto a la imputación de valores faltantes, se imputó el estado previo, lo cual demostró ser coherente con el funcionamiento del sistema. Por otro lado, se imputó la velocidad a “normal” como valor por defecto, habilitando el entrenamiento de modelos supervisados sin introducir supuestos arbitrarios.

La generación de características numéricas derivadas del texto aportó una capa adicional de información que complementa la representación semántica capturada por TF-IDF. Variables relacionadas con longitud, uso de signos, mayúsculas, cortesía y urgencia permitieron capturar aspectos de estilo e intención del comando. Estas características reflejan variabilidad real del lenguaje humano y contribuyen a la robustez del sistema frente a diferentes formas de expresión.
La correcta definición de las variables de entrada y de los objetivos permitió establecer con precisión qué información recibe el modelo y qué se espera que prediga, reduciendo ambigüedades y errores en etapas posteriores. La incorporación conjunta de contexto del sistema, texto normalizado y características numéricas derivadas del lenguaje enriqueció la representación de los comandos, evitando con ello que el modelo dependa únicamente del contenido textual. La validación de las dimensiones de los datasets confirmó la coherencia estructural entre features y targets, asegurando que cada observación cuente con sus etiquetas correspondientes y que los conjuntos de entrenamiento y prueba sean consistentes y aptos para el aprendizaje supervisado.

El diseño de un pipeline de preprocesamiento modular y reproducible permitió transformar adecuadamente variables categóricas, numéricas y textuales en una representación numérica uniforme y consistente. El tratamiento diferenciado según la naturaleza de cada variable, junto con el uso combinado de TF-IDF por palabras y por caracteres, dio lugar a una representación rica, robusta y tolerante a variaciones en el texto. La matriz final, altamente dispersa pero eficiente en términos computacionales, resultó especialmente adecuada para modelos lineales y métodos regularizados, dejando los datos correctamente preparados para un entrenamiento eficaz y estable.

En la sección de selección de features y modelado, se realizó una comparación para verificar si al “reducir” o “compactar” las features se mantiene o mejora el desempeño del modelo, usando un modelo base sin reducción y otro mediante TruncatedSVD. Los resultados muestran que el modelo base, apoyado en TF-IDF (palabras y caracteres) y un clasificador LinearSVC con balanceo de clases, alcanza un desempeño muy alto tanto en la predicción de "acción" como de "velocidad" (F1 score > 0.95). Mientras que el modelo con SVD logró reducir considerablemente la dimensionalidad y la complejidad del modelo, con una caída mínima el desempeño.

El análisis de correlación entre variables numéricas evidenció que no existen redundancias severas entre las características derivadas del texto, salvo la alta correlación esperable entre número de palabras y número de caracteres.

Finalmente, la aplicación de técnicas de extracción como PCA y Factor Analysis sobre el bloque numérico permitió comprobar que es posible resumir una parte sustancial de la información en un número reducido de componentes latentes. Con solo tres componentes principales se capturó alrededor del 62% de la varianza total.

En conjunto, el trabajo demuestra que una ingeniería de características bien fundamentada que combina limpieza, imputación lógica, enriquecimiento de variables, filtrado y extracción, es tan determinante como el algoritmo de clasificación elegido. Más allá de alcanzar métricas altas, el proceso seguido prioriza coherencia con el dominio, interpretabilidad y robustez, aspectos clave para un sistema de asistencia real donde la seguridad y la consistencia son fundamentales.
Este ejercicio sienta las bases para una futura etapa de despliegue y monitoreo, ya que las decisiones de imputación, representación y reducción de dimensionalidad favorecen la estabilidad del modelo ante cambios en la distribución de los comandos y permiten detectar degradaciones de desempeño de forma temprana.

Desde la perspectiva de CRISP-ML, el pipeline construido prioriza no solo el desempeño inicial, sino también la mantenibilidad, trazabilidad y capacidad de adaptación del sistema en escenarios reales de uso.